In [ ]:
# default_exp targetfeat

# targetfeat
> Module to generate target site features

In [ ]:
# export
import pandas as pd
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import warnings

In [ ]:
from rs3 import targetdata

In [ ]:
# export
def add_target_columns(design_df, transcript_id_col='Target Transcript',
                       cut_pos_col='Target Cut Length'):
    """Add ['AA Index' and 'Transcript Base'] to design df

    :param design_df: DataFrame
    :return: DataFrame
    """
    out_df = design_df.copy()
    out_df['AA Index'] = (out_df[cut_pos_col] - 1) // 3 + 1
    out_df['Transcript Base'] = out_df[transcript_id_col].str.split('.', expand=True)[0]
    return out_df

In [ ]:
design_df = pd.read_table('test_data/sgrna-designs.txt')
design_targ_df = add_target_columns(design_df)
assert 'AA Index' in design_targ_df.columns

## Position Features

The first feature class we consider is where the guide targets within the annotated transcript

In [ ]:
# export
def get_position_features(sg_df, id_cols):
    """Get  features ['Target Cut %', 'sense']

    :param sg_df: DataFrame
    :param id_cols: list
    :return: DataFrame
    """
    position_df = sg_df[id_cols + ['Target Cut %']].copy()
    position_df['sense'] = sg_df['Orientation'] == 'sense'
    return position_df

## Amino Acid Features

We calculate a set of features from the amino acid sequence around the cutsite itself

In [ ]:
aas = ['A', 'C', 'D', 'E', 'F',
       'G', 'H', 'I', 'K', 'L',
       'M', 'N', 'P', 'Q', 'R',
       'S', 'T', 'V', 'W', 'Y', '*']

In [ ]:
# export
def get_one_aa_frac(feature_dict, aa_sequence, aas):
    """Get fraction of single aa

    :param feature_dict: dict, feature dictionary
    :param aa_sequence: str, amino acid sequence
    :param aas: list, list of amino acids
    """
    for aa in aas:
        aa_frac = aa_sequence.count(aa) / len(aa_sequence)
        feature_dict[aa] = aa_frac

In [ ]:
one_aa_ft = {}
get_one_aa_frac(one_aa_ft, 'ACDG*-', aas)
assert one_aa_ft['A'] == 1/6
assert one_aa_ft['Q'] == 0

In [ ]:
# export
def get_aa_aromaticity(feature_dict, analyzed_seq):
    """Get fraction of aromatic amino acids in a sequence.

    Phe (F) + Trp (W) + Tyr (Y)

    :param feature_dict:
    :param analyzed_seq: ProteinAnalysis object
    """
    feature_dict['Aromaticity'] = analyzed_seq.aromaticity()


def get_aa_hydrophobicity(feature_dict, analyzed_seq):
    """Grand Average of Hydropathy

     The GRAVY value is calculated by adding the hydropathy value for each residue and dividing
     by the length of the sequence (Kyte and Doolittle; 1982). The larger the number, the more hydrophobic the
     amino acid

    :param feature_dict: dict
    :param analyzed_seq: ProteinAnalysis object
    """
    feature_dict['Hydrophobicity'] = analyzed_seq.gravy()


def get_aa_ip(feature_dict, analyzed_seq):
    """Get the Isoelectric Point of an amino acid sequence

    Charge of amino acid

    :param feature_dict: dict
    :param analyzed_seq: ProteinAnalysis object
    """
    feature_dict['Isoelectric Point'] = analyzed_seq.isoelectric_point()


def get_aa_secondary_structure(feature_dict, analyzed_seq):
    """Get the fraction of amion acids that tend to be in a helix, turn or sheet

    :param feature_dict: dict
    :param analyzed_seq: ProteinAnalysis object
    """
    feature_dict['Helix'], feature_dict['Turn'], feature_dict['Sheet'] = analyzed_seq.secondary_structure_fraction()


In [ ]:
aa_biochemical_fts1 = {}
get_aa_aromaticity(aa_biochemical_fts1, ProteinAnalysis('FWYA'))
aa_biochemical_fts2 = {}
get_aa_aromaticity(aa_biochemical_fts2, ProteinAnalysis('AAAA'))
assert aa_biochemical_fts1['Aromaticity'] > aa_biochemical_fts2['Aromaticity']

In [ ]:
# export
def featurize_aa_seqs(aa_sequences, features=None):
    """Get feature DataFrame for a list of amino acid sequences

    :param aa_sequences: list of str
    :param features: list or None
    :return: DataFrame
    """
    if features is None:
        features = ['Pos. Ind. 1mer', 'Hydrophobicity', 'Aromaticity',
                    'Isoelectric Point', 'Secondary Structure']
    aas = ['A', 'C', 'D', 'E', 'F',
           'G', 'H', 'I', 'K', 'L',
           'M', 'N', 'P', 'Q', 'R',
           'S', 'T', 'V', 'W', 'Y', '*']
    clean_aa_seqs = aa_sequences.str.replace('\*|-', '', regex=True)
    feature_dict_list = []
    for i, (aa_sequence, clean_sequence) in enumerate(zip(aa_sequences, clean_aa_seqs)):
        analyzed_seq = ProteinAnalysis(clean_sequence)
        feature_dict = {}
        if 'Pos. Ind. 1mer' in features:
            get_one_aa_frac(feature_dict, aa_sequence, aas)
        if 'Hydrophobicity' in features:
            get_aa_hydrophobicity(feature_dict, analyzed_seq)
        if 'Aromaticity' in features:
            get_aa_aromaticity(feature_dict, analyzed_seq)
        if 'Isoelectric Point' in features:
            get_aa_ip(feature_dict, analyzed_seq)
        if 'Secondary Structure' in features:
            get_aa_secondary_structure(feature_dict, analyzed_seq)
        feature_dict_list.append(feature_dict)
    feature_matrix = pd.DataFrame(feature_dict_list)
    feature_matrix.index = aa_sequences
    return feature_matrix

In [ ]:
ft_dict_df = featurize_aa_seqs(pd.Series(['ACDG*-', 'CDG*--', 'LLLLLL']))
assert ft_dict_df.loc['LLLLLL', 'Hydrophobicity'] == ft_dict_df['Hydrophobicity'].max()

In [ ]:
# export
def get_amino_acid_features(sg_designs, aa_seq_df, width, features, id_cols,
                            transcript_base_col='Transcript Base',
                            target_transcript_col='Target Transcript',
                            aa_index_col='AA Index'):
    """Featurize amino acid sequences

    :param sg_designs: DataFrame
    :param aa_seq_df: DataFrame, Transcript Base and (AA) seq
    :param width: int, length on each side of the cut site
    :param feature: list
    :param transcript_base_col: str
    :param target_transcript_col: str
    :param aa_index_col: str
    :return: DataFrame
    """
    sg_aas = (aa_seq_df.merge(sg_designs[list(set(id_cols +
                                                  [target_transcript_col, transcript_base_col, aa_index_col]))],
                              how='inner',
                              on=[target_transcript_col, transcript_base_col]))
    padding = '-' * (width + 1)
    sg_aas['extended_seq'] = padding + sg_aas['seq'] + '*' + padding
    sg_aas['AA Index pad'] = sg_aas[aa_index_col] + width + 1
    # One-indexed
    sg_aas['seq_start'] = sg_aas['AA Index pad'] - width
    sg_aas['seq_end'] = sg_aas['AA Index pad'] + width - 1
    # Zero-indexed for python
    sg_aas['AA Subsequence'] = sg_aas.apply(lambda row: row['extended_seq'][(row['seq_start'] - 1):row['seq_end']],
                                            axis=1)
    # filter out sequences without the canonical amino acids
    aa_set = set('ARNDCQEGHILKMFPSTWYV*-')
    filtered_sg_aas = (sg_aas[sg_aas['AA Subsequence'].apply(lambda s: set(s) <= aa_set)]
                       .reset_index(drop=True))
    filtered_diff = (sg_aas.shape[0] - filtered_sg_aas.shape[0])
    if filtered_diff > 0:
        warnings.warn('Ignored ' + str(filtered_diff) + ' amino acid sequences with non-canonical amino acids')
    aa_features = featurize_aa_seqs(filtered_sg_aas['AA Subsequence'], features=features)
    aa_features_annot = pd.concat([filtered_sg_aas[id_cols + ['AA Subsequence']]
                                   .reset_index(drop=True),
                                   aa_features.reset_index(drop=True)], axis=1)
    return aa_features_annot


In [ ]:
aa_seq_df = targetdata.build_transcript_aa_seq_df(design_targ_df, n_jobs=2)

def get_rev_comp(sgrna):
    """Get reverse compliment of a guide"""
    nt_map = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
    rev_comp = ''
    for nt in sgrna:
        rev_comp += nt_map[nt]
    rev_comp = rev_comp[::-1]
    return rev_comp

aa_features = get_amino_acid_features(design_targ_df, aa_seq_df, width=10,
                                      features=['Pos. Ind. 1mer', 'Pos. Ind. 2mer', 'Pos. Dep. 1mer',
                                                'Hydrophobicity', 'Aromaticity',
                                                'Isoelectric Point', 'Secondary Structure'],
                                      id_cols=['sgRNA Context Sequence', 'Target Cut Length',
                                               'Target Transcript', 'Orientation'])
assert (aa_features['AA Subsequence'].str.len() == 20).all()

Getting amino acid sequences


100%|██████████| 4/4 [00:02<00:00,  1.79it/s]


In [ ]:
codon_map_df = pd.read_csv('test_data/codon_map.csv')
codon_map = pd.Series(codon_map_df['Amino Acid'].values, index=codon_map_df['Codon']).to_dict()
row = aa_features.sample(1, random_state=1).iloc[0, :]
subseq = row['AA Subsequence']
context = row['sgRNA Context Sequence']
rc_context = get_rev_comp(context)
translations = dict()
rc_translations = dict()
for i in [0, 1, 2]:
    translations[i] = ''.join([codon_map[context[j:j+3]] for j in range(i, len(context), 3)
                               if (j + 3) <= len(context)])
    rc_translations[i] = ''.join([codon_map[rc_context[j:j+3]] for j in range(i, len(rc_context), 3)
                                  if (j + 3) <= len(rc_context)])
assert ((translations[0] in subseq) or (translations[1] in subseq) or (translations[2] in subseq) or
        (rc_translations[0] in subseq) or (rc_translations[1] in subseq) or (rc_translations[2] in subseq))

## Protein Domain Features

In [ ]:
#export
def get_protein_domain_features(sg_design_df, protein_domains, sources, id_cols,
                                transcript_base_col='Transcript Base',
                                aa_index_col='AA Index',
                                domain_type_col='type',
                                domain_start_col='start',
                                domain_end_col='end'):
    """Get binary dataframe of protein domains

    :param sg_design_df: DataFrame, with columns [transcript_base_col, aa_index_col]
    :param protein_domains: DataFrame, with columns [transcript_base_col, domain_type_col]
    :param sources: list. list of database types to include
    :param id_cols: list
    :param transcript_base_col: str
    :param aa_index_col: str
    :param domain_type_col: str
    :param domain_start_col: str
    :param domain_end_col: str
    :return: DataFrame, with binary features for protein domains
    """
    if sources is None:
        sources = ['Pfam', 'PANTHER', 'HAMAP', 'SuperFamily', 'TIGRfam', 'ncoils', 'Gene3D',
                   'Prosite_patterns', 'Seg', 'SignalP', 'TMHMM', 'MobiDBLite',
                   'PIRSF', 'PRINTS', 'Smart', 'Prosite_profiles']  # exclude sifts
    protein_domains = protein_domains[protein_domains[domain_type_col].isin(sources)]
    clean_designs = sg_design_df[list(set(id_cols + [transcript_base_col, aa_index_col]))].copy()
    designs_domains = clean_designs.merge(protein_domains,
                                          how='inner', on=transcript_base_col)
    # Note - not every sgRNA will be present in the feature df
    filtered_domains = (designs_domains[designs_domains[aa_index_col].between(designs_domains[domain_start_col],
                                                                              designs_domains[domain_end_col])]
                        .copy())
    filtered_domains = filtered_domains[id_cols + [domain_type_col]].drop_duplicates()
    filtered_domains['present'] = 1
    domain_feature_df = (filtered_domains.pivot_table(values='present',
                                                      index=id_cols,
                                                      columns='type', fill_value=0)
                         .reset_index())
    # Ensure all domain columns are present for testing
    full_column_df = pd.DataFrame(columns=id_cols + sources, dtype=int)  # empty
    domain_feature_df = pd.concat([full_column_df, domain_feature_df]).fillna(0)
    domain_feature_df[sources] = domain_feature_df[sources].astype(int)
    return domain_feature_df

In [ ]:
domain_df = targetdata.build_translation_overlap_df(aa_seq_df['id'].unique(), n_jobs=2)
protein_domain_feature_df = get_protein_domain_features(design_targ_df, domain_df, sources=None,
                                                        id_cols=['sgRNA Context Sequence', 'Target Cut Length',
                                                                 'AA Index', 'Target Transcript', 'Orientation'])

Getting protein domains


100%|██████████| 200/200 [00:52<00:00,  3.79it/s]


In [ ]:
assert protein_domain_feature_df.loc[protein_domain_feature_df['sgRNA Context Sequence'] == 'AAAAGAGCCATGAATCTAAACATCAGGAAT',
                                     ['PANTHER', 'ncoils', 'Seg', 'MobiDBLite']].sum(axis=1).values[0] == 4

## Combining target features

We'll combine, the position, amino acid and domain feature matrices into a single target feature matrix

In [ ]:
# export
def build_target_feature_df(sg_designs, features=None,
                            aa_seq_df=None, aa_width=16, aa_features=None,
                            protein_domain_df=None, protein_domain_sources=None,
                            id_cols=None):
    """Build the feature matrix for the sgRNA target site

    :param sg_designs: DataFrame
    :param features: list
    :param aa_seq_df: DataFrame
    :param aa_width: int
    :param aa_features: list
    :param protein_domain_df: DataFrame
    :param protein_domain_sources: list or None. Defaults to all sources except Sifts
    :return: (feature_df, feature_list)
        feature_df: DataFrame
        feature_list: list
    """
    if features is None:
        features = ['position', 'aa', 'domain']
    if id_cols is None:
        id_cols = ['sgRNA Context Sequence', 'Target Cut Length',
                   'Target Transcript', 'Orientation']
    if sg_designs[id_cols].drop_duplicates().shape[0] != sg_designs.shape[0]:
        raise ValueError('id_cols must uniquely identify rows of the design dataframe')
    design_df = add_target_columns(sg_designs)
    feature_df_dict = dict()
    if 'position' in features:
        position_features = get_position_features(design_df, id_cols)
        feature_df_dict['position'] = position_features
    if 'domain' in features:
        domain_features = get_protein_domain_features(design_df, protein_domain_df, protein_domain_sources, id_cols)
        feature_df_dict['domain'] = domain_features
    if 'aa' in features:
        aa_features = get_amino_acid_features(design_df, aa_seq_df, aa_width, aa_features, id_cols)
        feature_df_dict['aa'] = aa_features
    feature_df = design_df[id_cols]
    for key, df in feature_df_dict.items():
        feature_df = pd.merge(feature_df, df, how='left', on=id_cols)
    full_feature_list = list(feature_df.columns)
    remove_cols = id_cols + ['Transcript Base', 'AA Index', 'AA Subsequence']
    for col in remove_cols:
        if col in full_feature_list:
            full_feature_list.remove(col)
    return feature_df, full_feature_list

In [ ]:
feature_df, feature_list = build_target_feature_df(sg_designs=design_df,
                                                   aa_seq_df=aa_seq_df,
                                                   protein_domain_df=domain_df)
assert len(feature_list) > 20
assert feature_df.shape[0] == design_df.shape[0]